Example to handle Super-Kamiokande specific format from the official data release:
- The effective areas are the ones published in https://doi.org/10.5281/zenodo.4724822.
- The expected background is extracted from https://doi.org/10.3847/1538-4357/ac0d5a.
- The observed number of events is arbitrarily fixed to 0 (consistent with O3a follow-up results published in https://doi.org/10.3847/1538-4357/ac0d5a).

In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import astropy.time
import h5py
from scipy.interpolate import RegularGridInterpolator

import momenta.utils.flux as flux
from momenta.io import GW, NuDetector, Parameters
from momenta.io.neutrinos_irfs import EffectiveAreaAltitudeDep
from momenta.io.neutrinos import BackgroundFixed
from momenta.stats.run import run_ultranest
from momenta.stats.constraints import get_limits

We define a new class inheriting from `EffectiveAreaAltitudeDep`, as the SK effective area are provided in zenith and energy bins.

In [2]:
class EffectiveAreaSK(EffectiveAreaAltitudeDep):
    def __init__(self, filename: str, samplename: str, time: astropy.time.Time):
        super().__init__()
        self.read(filename, samplename)
        self.set_location(time, 36.425634, 137.310340)

    def read(self, filename: str, samplename: str):
        with h5py.File(filename, "r") as f:
            bins_logenergy = f["bins_logenergy"][:]
            bins_altitude = f["bins_altitude"][:]
            aeff = f[f"aeff_{samplename}"][:]
        self.func = RegularGridInterpolator((bins_logenergy, bins_altitude), aeff, bounds_error=False, fill_value=0)

In [3]:
parameters = Parameters("input_files/config.yaml")
parameters.set_flux(flux.FluxFixedPowerLaw(0.1, 1e6, 2))
parameters.nside = 8

Here, we consider a O3 binary merger as our source of interest. We define the source object accordingly.

In [4]:
gw = GW(
    path_to_fits="input_files/gw_catalogs/GW190412/GW190412_PublicationSamples.fits",
    path_to_samples="input_files/gw_catalogs/GW190412/GW190412_subset.h5"
)
gw.set_parameters(parameters)  # this is needed to speciffy parameters used in GW methods

[2025-11-04 10:04:36,629:momenta:INFO] [GW] Fits is loaded from the file GW190412_PublicationSamples.fits
[2025-11-04 10:04:36,629:momenta:INFO] [GW] Samples are loaded from the file GW190412_subset.h5


From there, we can define all the objects used by `momenta`, namely the `NuDetector` object, and the related instrumental response functions

In [5]:
sk = NuDetector(
    {
        "name": "Super-Kamiokande",
        "samples": ["FC", "PC", "UPMU"],
        "errors": {"acceptance": 0, "acceptance_corr": 1}
    }
)

sk.set_effective_areas([EffectiveAreaSK("input_files/effarea_superk.h5", s.name, gw.utc) for s in sk.samples])

nobs = [0, 0, 0]
nbkg = [0.112, 0.007, 0.016]
bkg = [BackgroundFixed(b) for b in nbkg]
sk.set_observations(nobs, bkg)

[2025-11-04 10:04:36,689:momenta:INFO] [NuDetector] Object is loaded from a dictionary object.


The analysis can then be run with a single command line.

In [6]:
_, result = run_ultranest(sk, gw, parameters)
limits = get_limits(result, CL=0.90)
print(f"90% upper limit on flux normalisation: {limits['fluxnorm0']:.2e} /GeV/cm²")

90% upper limit on flux normalisation: 1.12e+02 /GeV/cm²
